In [ ]:
import gdown
import zipfile
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import shutil
import numpy as np
from tqdm import tqdm
import pickle
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
from torch.optim.lr_scheduler import MultiStepLR, ExponentialLR
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter


In [ ]:
# Step 1: Clone the GitHub repository
!git clone https://github.com/timesler/facenet-pytorch.git

# Step 2: Append the repository's directory to Python's system path
import sys
sys.path.append('/content/facenet-pytorch')

Cloning into 'facenet-pytorch'...
remote: Enumerating objects: 1318, done.
remote: Counting objects: 100% (337/337), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 1318 (delta 290), reused 277 (delta 277), pack-reused 981
Receiving objects: 100% (1318/1318), 23.18 MiB | 25.63 MiB/s, done.
Resolving deltas: 100% (656/656), done.


In [ ]:
from models.mtcnn import MTCNN, fixed_image_standardization
from models.inception_resnet_v1 import InceptionResnetV1
import models.utils.training as training

In [ ]:
dataset = "large"
# Get Data
file_id_aux = '1ycNWvvAZP5NxQVRP-kqejy1LZGQFFI3B'

if(dataset == "large"):
  file_id_faces = '15Mni2hW0okL9oyA94BTxvSROCLsx0x5a'

# Construct the Google Drive URL for the file
url_faces = f'https://drive.google.com/uc?id={file_id_faces}'
url_aux = f'https://drive.google.com/uc?id={file_id_aux}'

# Specify the destination filename
output_filename_faces = 'faces.zip'
output_filename_aux = 'aux.zip'

# Download the file
gdown.download(url_faces, output_filename_faces, quiet=False)
gdown.download(url_aux, output_filename_aux, quiet=False)

# Unzip the file
with zipfile.ZipFile(output_filename_faces, 'r') as zip_ref:
    zip_ref.extractall("faces")
!rm -rf /content/faces.zip
with zipfile.ZipFile(output_filename_aux, 'r') as zip_ref:
    zip_ref.extractall("aux")
!rm -rf /content/aux.zip

Downloading...
From (original): https://drive.google.com/uc?id=15Mni2hW0okL9oyA94BTxvSROCLsx0x5a
From (redirected): https://drive.google.com/uc?id=15Mni2hW0okL9oyA94BTxvSROCLsx0x5a&confirm=t&uuid=25ff3704-e44d-4972-a6d3-b547fc290584
To: /content/faces.zip
100%|██████████| 314M/314M [00:03<00:00, 101MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1ycNWvvAZP5NxQVRP-kqejy1LZGQFFI3B
To: /content/aux.zip
100%|██████████| 576k/576k [00:00<00:00, 80.9MB/s]


In [ ]:
data_dir = '/content/faces/train'

batch_size = 32
epochs = 10
workers = 0 if os.name == 'nt' else 2

Bellow uses code from: https://github.com/timesler/facenet-pytorch

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


Bellow uses code from: https://github.com/timesler/facenet-pytorch

In [ ]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((160, 160)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]

loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

Bellow uses code from: https://github.com/timesler/facenet-pytorch

In [ ]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

Bellow uses code from: https://github.com/timesler/facenet-pytorch

In [ ]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 7])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir, transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

Bellow uses code from: https://github.com/timesler/facenet-pytorch

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

Bellow uses code from: https://github.com/timesler/facenet-pytorch

In [ ]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |   428/428  | loss:    4.7205 | fps:  576.3453 | acc:    0.0086   

Epoch 1/10
----------
Train |  1709/1709 | loss:    2.7480 | fps:  213.7493 | acc:    0.3908   
Valid |   428/428  | loss:    2.0766 | fps:  556.3334 | acc:    0.5614   

Epoch 2/10
----------
Train |  1709/1709 | loss:    1.8017 | fps:  209.3230 | acc:    0.6185   
Valid |   428/428  | loss:    1.6517 | fps:  576.4152 | acc:    0.6634   

Epoch 3/10
----------
Train |  1709/1709 | loss:    1.4243 | fps:  211.2684 | acc:    0.6963   
Valid |   428/428  | loss:    1.5630 | fps:  561.0714 | acc:    0.6781   

Epoch 4/10
----------
Train |  1709/1709 | loss:    1.1463 | fps:  212.0411 | acc:    0.7487   
Valid |   428/428  | loss:    1.4329 | fps:  570.6538 | acc:    0.7117   

Epoch 5/10
----------
Train |  1709/1709 | loss:    0.8942 | fps:  210.9996 | acc:    0.7940   
Valid |   428/428  | loss:    1.3906 | fps:  562.3038 | acc:    0.7328   

Epoch 6/10
----------
Train |  1709/1709 | loss: 

In [ ]:
writer.close()
# Step 2: Load the TensorBoard notebook extension
%load_ext tensorboard

# Step 3: Start TensorBoard within the notebook using the same directory as the SummaryWriter
%tensorboard --logdir runs

In [ ]:
test = next(iter(val_loader))

input = test[0][0].unsqueeze(0).to(device)
# print(input.shape)
resnet.eval()
output = resnet(input)

In [ ]:
probabilities = F.softmax(output, dim=1)


max_prob, index = torch.max(probabilities, dim=1)

# Convert the max probability to a percentage
percentage = max_prob.item() * 100

print(f"Predicted class index: {index.item()}")
print(f"Confidence percentage: {percentage:.2f}%")

Predicted class index: 59
Confidence percentage: 57.70%


Test Face net

# Imports

In [ ]:
import gdown
import zipfile
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import shutil
import numpy as np
from tqdm import tqdm

# !pip install facenet-pytorch

# from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
from PIL import Image

import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

In [ ]:
# Step 1: Clone the GitHub repository
!git clone https://github.com/timesler/facenet-pytorch.git

# Step 2: Append the repository's directory to Python's system path
import sys
sys.path.append('/content/facenet-pytorch')

Cloning into 'facenet-pytorch'...
remote: Enumerating objects: 1318, done.
remote: Counting objects: 100% (337/337), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 1318 (delta 290), reused 277 (delta 277), pack-reused 981
Receiving objects: 100% (1318/1318), 23.18 MiB | 22.37 MiB/s, done.
Resolving deltas: 100% (656/656), done.


In [ ]:
from models.mtcnn import MTCNN, fixed_image_standardization
from models.inception_resnet_v1 import InceptionResnetV1
import models.utils.training as training

# Dataset

## Get Raw Faces

In [ ]:
# Get Data
file_id_aux = '1ycNWvvAZP5NxQVRP-kqejy1LZGQFFI3B'

file_id_faces = '1Pz3eoEObO5-TFq5lu6PU9vEr70YbvRqp'

# Construct the Google Drive URL for the file
url_faces = f'https://drive.google.com/uc?id={file_id_faces}'
url_aux = f'https://drive.google.com/uc?id={file_id_aux}'

# Specify the destination filename
output_filename_faces = 'faces.zip'
output_filename_aux = 'aux.zip'

# Download the file
gdown.download(url_faces, output_filename_faces, quiet=False)
gdown.download(url_aux, output_filename_aux, quiet=False)

# Unzip the file
with zipfile.ZipFile(output_filename_faces, 'r') as zip_ref:
    zip_ref.extractall("faces")
!rm -rf /content/faces.zip
with zipfile.ZipFile(output_filename_aux, 'r') as zip_ref:
    zip_ref.extractall("aux")
!rm -rf /content/aux.zip

Downloading...
From (original): https://drive.google.com/uc?id=1Pz3eoEObO5-TFq5lu6PU9vEr70YbvRqp
From (redirected): https://drive.google.com/uc?id=1Pz3eoEObO5-TFq5lu6PU9vEr70YbvRqp&confirm=t&uuid=b1549082-f240-4c94-99d0-abf0f8ead6f4
To: /content/faces.zip
100%|██████████| 996M/996M [00:10<00:00, 94.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ycNWvvAZP5NxQVRP-kqejy1LZGQFFI3B
To: /content/aux.zip
100%|██████████| 576k/576k [00:00<00:00, 8.06MB/s]


In [ ]:
image_directory = "/content/faces/test"
csv_file_path = "/content/aux/train.csv"

# List all files in the directory
# image_files = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]
image_files = os.listdir(image_directory)
image_files = [os.path.join(image_directory, f) for f in image_files]

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device, keep_all=True
)

for image in tqdm(image_files):

  with Image.open(image) as img:

    # Convert the image to RGB
    img = img.convert("RGB")

    # Determine the dimensions to achieve a square image
    width, height = img.size
    max_side = max(width, height)

    # Calculate padding for the top, bottom, left, and right
    left = (max_side - width) // 2
    right = max_side - width - left
    top = (max_side - height) // 2
    bottom = max_side - height - top

    padding_color=(0, 0, 0)
    # Add padding to make the image square
    img_with_padding = Image.new(img.mode, (max_side, max_side), padding_color)
    img_with_padding.paste(img, (left, top))

    resized_img = img_with_padding.resize((512,512), Image.ANTIALIAS)


    mtcnn(img, save_path=image)

  # # Save or display the resulting image
  # resized_img.save(image)

Running on device: cuda:0


  0%|          | 0/4977 [00:00<?, ?it/s]<ipython-input-7-4be38e0ad3c1>:32: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_img = img_with_padding.resize((512,512), Image.ANTIALIAS)
 46%|████▋     | 2313/4977 [07:48<07:06,  6.24it/s]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 4977/4977 [17:29<00:00,  4.74it/s]


In [ ]:
!zip -r /content/file.zip /content/faces/test


In [ ]:
shutil.move('/content/file.zip', '/content/drive/MyDrive')

In [ ]:
!rm -rf /content/aux

## Download Cropped Faces

In [ ]:
# Get Data
file_id_aux = '1ycNWvvAZP5NxQVRP-kqejy1LZGQFFI3B'

file_id_faces = '109dES3CC5PSg_PUidudv2eoecA0VkmZ8'

# Construct the Google Drive URL for the file
url_faces = f'https://drive.google.com/uc?id={file_id_faces}'
url_aux = f'https://drive.google.com/uc?id={file_id_aux}'

# Specify the destination filename
output_filename_faces = 'faces.zip'
output_filename_aux = 'aux.zip'

# Download the file
gdown.download(url_faces, output_filename_faces, quiet=False)
gdown.download(url_aux, output_filename_aux, quiet=False)

# Unzip the file
with zipfile.ZipFile(output_filename_faces, 'r') as zip_ref:
    zip_ref.extractall("faces")
!rm -rf /content/faces.zip
with zipfile.ZipFile(output_filename_aux, 'r') as zip_ref:
    zip_ref.extractall("aux")
!rm -rf /content/aux.zip

Downloading...
From (original): https://drive.google.com/uc?id=109dES3CC5PSg_PUidudv2eoecA0VkmZ8
From (redirected): https://drive.google.com/uc?id=109dES3CC5PSg_PUidudv2eoecA0VkmZ8&confirm=t&uuid=ccd8f63b-56f4-4537-8cb2-8e50e2f51be2
To: /content/faces.zip
100%|██████████| 35.0M/35.0M [00:00<00:00, 109MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ycNWvvAZP5NxQVRP-kqejy1LZGQFFI3B
To: /content/aux.zip
100%|██████████| 576k/576k [00:00<00:00, 116MB/s]


In [ ]:
image_directory = "/content/faces/test"
csv_file_path = "/content/aux/train.csv"

# List all files in the directory
# image_files = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]
image_files = os.listdir(image_directory)
image_files = [os.path.join(image_directory, f) for f in image_files]

## If any files didnt get cropped crop them now



In [ ]:
count = 0

for image in tqdm(image_files):

  with Image.open(image) as img:

    # Determine the dimensions to achieve a square image
    width, height = img.size

    if width != 160 or height != 160:

      count = count + 1

      resized_img = img.resize((160,160), Image.ANTIALIAS)
      resized_img = resized_img.convert('RGB') # Convert to RGB mode
      resized_img.save(image)

print(count, count / 5000)

  0%|          | 0/7326 [00:00<?, ?it/s]<ipython-input-9-95db99a8888f>:14: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_img = img.resize((160,160), Image.ANTIALIAS)
100%|██████████| 7326/7326 [00:01<00:00, 4286.17it/s]

103 0.0206


## Sort the Image Files

In [ ]:

# Custom sort key function to extract the numerical part
def sort_key(filename):
    # Extract the number from the filename and convert to int
    if "_" in filename:
      return int(filename.split("_")[0].split("/")[-1])
    else:
      return int(filename.split('/')[-1].split('.')[0])

# Sort the list using the custom key function
sorted_files = sorted(image_files, key=sort_key)

# Now sorted_files is in the numerical order based on the image file name
print(sorted_files)

['/content/faces/test/0.jpg', '/content/faces/test/1_3.jpg', '/content/faces/test/1_2.jpg', '/content/faces/test/1.jpg', '/content/faces/test/2.jpg', '/content/faces/test/3.jpg', '/content/faces/test/4.jpg', '/content/faces/test/5.jpg', '/content/faces/test/6.jpg', '/content/faces/test/7.jpg', '/content/faces/test/8.jpg', '/content/faces/test/9.jpg', '/content/faces/test/9_2.jpg', '/content/faces/test/10.jpg', '/content/faces/test/11.jpg', '/content/faces/test/12.jpg', '/content/faces/test/13.jpg', '/content/faces/test/14.jpg', '/content/faces/test/15_4.jpg', '/content/faces/test/15_3.jpg', '/content/faces/test/15.jpg', '/content/faces/test/15_5.jpg', '/content/faces/test/15_2.jpg', '/content/faces/test/16.jpg', '/content/faces/test/17.jpg', '/content/faces/test/18.jpg', '/content/faces/test/19.jpg', '/content/faces/test/19_3.jpg', '/content/faces/test/19_2.jpg', '/content/faces/test/20.jpg', '/content/faces/test/21.jpg', '/content/faces/test/22.jpg', '/content/faces/test/23_2.jpg', '/

In [ ]:
# List of filenames
filenames = sorted_files

# Initialize a dictionary to hold the groups
grouped_files = {}

# Loop through each filename
for filename in filenames:
    # Extract the starting number (everything before the first underscore)
    if "_" in filename:
      start_number = int(filename.split("_")[0].split("/")[-1])
    else:
      start_number = int(filename.split('/')[-1].split('.')[0])

    # Check if the start number is already a key in the dictionary
    if start_number not in grouped_files:
        # If not, add it with the current filename in a new list
        grouped_files[start_number] = [filename]
    else:
        # If it is, append the current filename to the existing list
        grouped_files[start_number].append(filename)

# Print the grouped files
for start_number, files in grouped_files.items():
    print(f"Group {start_number}: {files}")

Group 0: ['/content/faces/test/0.jpg']
Group 1: ['/content/faces/test/1_3.jpg', '/content/faces/test/1_2.jpg', '/content/faces/test/1.jpg']
Group 2: ['/content/faces/test/2.jpg']
Group 3: ['/content/faces/test/3.jpg']
Group 4: ['/content/faces/test/4.jpg']
Group 5: ['/content/faces/test/5.jpg']
Group 6: ['/content/faces/test/6.jpg']
Group 7: ['/content/faces/test/7.jpg']
Group 8: ['/content/faces/test/8.jpg']
Group 9: ['/content/faces/test/9.jpg', '/content/faces/test/9_2.jpg']
Group 10: ['/content/faces/test/10.jpg']
Group 11: ['/content/faces/test/11.jpg']
Group 12: ['/content/faces/test/12.jpg']
Group 13: ['/content/faces/test/13.jpg']
Group 14: ['/content/faces/test/14.jpg']
Group 15: ['/content/faces/test/15_4.jpg', '/content/faces/test/15_3.jpg', '/content/faces/test/15.jpg', '/content/faces/test/15_5.jpg', '/content/faces/test/15_2.jpg']
Group 16: ['/content/faces/test/16.jpg']
Group 17: ['/content/faces/test/17.jpg']
Group 18: ['/content/faces/test/18.jpg']
Group 19: ['/content

In [ ]:
def sort_key(filename):
    # Extract the number from the filename and convert to int
    if "_" in filename:
      return int(filename.split("_")[-1].split(".")[0])
    else:
      return 1

# Sort the list using the custom key function

for start_number, files in grouped_files.items():
  grouped_files[start_number] = sorted(files, key=sort_key)

# Print the grouped files
for start_number, files in grouped_files.items():
    print(f"Group {start_number}: {files}")


Group 0: ['/content/faces/test/0.jpg']
Group 1: ['/content/faces/test/1.jpg', '/content/faces/test/1_2.jpg', '/content/faces/test/1_3.jpg']
Group 2: ['/content/faces/test/2.jpg']
Group 3: ['/content/faces/test/3.jpg']
Group 4: ['/content/faces/test/4.jpg']
Group 5: ['/content/faces/test/5.jpg']
Group 6: ['/content/faces/test/6.jpg']
Group 7: ['/content/faces/test/7.jpg']
Group 8: ['/content/faces/test/8.jpg']
Group 9: ['/content/faces/test/9.jpg', '/content/faces/test/9_2.jpg']
Group 10: ['/content/faces/test/10.jpg']
Group 11: ['/content/faces/test/11.jpg']
Group 12: ['/content/faces/test/12.jpg']
Group 13: ['/content/faces/test/13.jpg']
Group 14: ['/content/faces/test/14.jpg']
Group 15: ['/content/faces/test/15.jpg', '/content/faces/test/15_2.jpg', '/content/faces/test/15_3.jpg', '/content/faces/test/15_4.jpg', '/content/faces/test/15_5.jpg']
Group 16: ['/content/faces/test/16.jpg']
Group 17: ['/content/faces/test/17.jpg']
Group 18: ['/content/faces/test/18.jpg']
Group 19: ['/content

In [ ]:
print(len(grouped_files))

4977


# Run Inference

Bellow uses code from: https://github.com/timesler/facenet-pytorch

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


Bellow uses code from: https://github.com/timesler/facenet-pytorch

In [ ]:
resnet = torch.load('/content/model.pth')

In [ ]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=100
).to(device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

Bellow uses code from: https://github.com/timesler/facenet-pytorch

In [ ]:
resnet.load_state_dict(torch.load('/content/resnet.pt'))
resnet.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with Image.open('/content/faces/test/64.jpg') as img:
  convert_tensor = transforms.ToTensor()
  tensor = convert_tensor(img)
  batch_tensor = tensor.unsqueeze(0).to(device)
  result = resnet(batch_tensor).detach().cpu()


print(result.shape)
max_val, max_idx = torch.max(result[0], 0)

# Print the maximum value and its original index in the tensor
print("Maximum value:", max_val.item())
print("Index of the maximum value in the flattened tensor:", max_idx.item())


torch.Size([1, 100])
Maximum value: 16.63741683959961
Index of the maximum value in the flattened tensor: 43


In [ ]:
df = pd.read_csv('/content/aux/category.csv')

# print(list(df['Category']))

correct_list = list(df['Category'])

incorrect_list = sorted(correct_list)

def find_correct_index(incorrect_list, correct_list, index):

    # Create a mapping from element to index for the correct list
    correct_mapping = {element: idx for idx, element in enumerate(correct_list)}

    # Find the element at the given index in the incorrect list
    element = incorrect_list[index]

    # Find the index of this element in the correct list using the mapping
    correct_index = correct_mapping[element]

    return correct_index

print(find_correct_index(incorrect_list, correct_list, 1))

47


In [ ]:
# def run_image(file_path):
#   with Image.open(file_path) as img:
#     convert_tensor = transforms.ToTensor()
#     tensor = convert_tensor(img)
#     batch_tensor = tensor.unsqueeze(0).to(device)
#     result = resnet(batch_tensor).detach().cpu()

#     max_val, max_idx = torch.max(result[0], 0)

#     return(max_idx.item())



In [ ]:
import torch.nn.functional as F
def run_image(file_path):
  with Image.open(file_path) as img:
    convert_tensor = transforms.ToTensor()
    tensor = convert_tensor(img)
    batch_tensor = tensor.unsqueeze(0).to(device)
    result = resnet(batch_tensor).detach().cpu()


    probabilities = F.softmax(result, dim=1)
    max_prob, index = torch.max(probabilities, dim=1)

    return(index.item(), max_prob)

In [ ]:
predictions = []
for start_number, files in tqdm(grouped_files.items()):
    max_prob = 0
    ID = None
    i = 0
    while i < len(files) and i < 3:
      index, prob = run_image(files[i])
      if prob > max_prob:
        max_prob = prob
        ID = index
      i += 1
    predictions.append((start_number, ID))
    # print(f"Group {start_number}: {ID}, {max_prob}")

100%|██████████| 4977/4977 [25:16<00:00,  3.28it/s]


In [ ]:
print(predictions[:10])
preds=[]
for predictions in predictions:
  preds.append(predictions[1])
print(preds)

[(0, 34), (1, 34), (2, 9), (3, 6), (4, 16), (5, 2), (6, 32), (7, 5), (8, 73), (9, 20)]
[34, 34, 9, 6, 16, 2, 32, 5, 73, 20, 50, 62, 92, 74, 85, 86, 19, 14, 10, 37, 67, 48, 83, 5, 68, 15, 56, 79, 9, 45, 47, 73, 33, 11, 55, 39, 40, 70, 53, 19, 94, 99, 37, 57, 88, 34, 75, 8, 14, 19, 1, 73, 50, 28, 50, 25, 5, 41, 94, 18, 37, 64, 72, 12, 43, 0, 39, 58, 74, 60, 16, 66, 36, 5, 66, 14, 45, 19, 12, 51, 76, 39, 3, 52, 4, 49, 58, 48, 65, 61, 3, 84, 31, 18, 26, 27, 29, 81, 66, 32, 7, 41, 69, 50, 73, 53, 53, 74, 69, 36, 77, 35, 80, 0, 28, 40, 97, 27, 51, 21, 58, 72, 1, 27, 60, 90, 42, 25, 86, 95, 75, 58, 50, 14, 65, 6, 30, 17, 69, 99, 5, 32, 59, 55, 18, 99, 98, 55, 41, 1, 91, 80, 98, 39, 80, 54, 3, 37, 84, 51, 80, 17, 50, 20, 10, 35, 99, 50, 16, 8, 28, 6, 66, 2, 90, 57, 55, 31, 22, 75, 56, 0, 41, 34, 22, 21, 60, 50, 3, 40, 86, 28, 73, 56, 51, 38, 34, 46, 36, 71, 74, 59, 63, 19, 80, 37, 97, 27, 89, 0, 77, 0, 32, 28, 1, 37, 35, 58, 82, 96, 50, 3, 66, 5, 48, 83, 52, 5, 15, 53, 27, 63, 35, 18, 73, 18, 

In [ ]:
preds = []
for img in tqdm(sorted_files):
  preds.append(run_image(img))

print(preds)


100%|██████████| 4977/4977 [02:04<00:00, 39.87it/s]

[88, 34, 9, 6, 16, 85, 32, 5, 73, 20, 50, 62, 92, 74, 85, 18, 19, 14, 10, 37, 67, 52, 83, 5, 68, 15, 59, 95, 9, 13, 47, 73, 33, 11, 40, 39, 40, 6, 53, 19, 76, 99, 37, 57, 89, 34, 75, 6, 14, 19, 1, 73, 98, 77, 28, 25, 5, 41, 94, 18, 37, 64, 72, 44, 43, 0, 39, 77, 74, 46, 16, 80, 36, 5, 35, 57, 28, 19, 12, 51, 82, 39, 3, 52, 32, 49, 7, 48, 65, 61, 3, 84, 31, 37, 26, 27, 29, 81, 66, 32, 7, 41, 69, 50, 63, 50, 53, 74, 69, 34, 77, 35, 80, 0, 54, 40, 97, 76, 51, 21, 58, 72, 1, 27, 60, 51, 42, 25, 24, 95, 75, 58, 50, 73, 65, 6, 30, 17, 69, 54, 5, 32, 59, 55, 18, 22, 98, 29, 29, 1, 17, 80, 98, 75, 80, 72, 12, 88, 84, 51, 22, 17, 50, 20, 10, 35, 99, 21, 13, 8, 28, 50, 66, 17, 90, 57, 29, 31, 68, 75, 63, 34, 79, 36, 22, 21, 16, 45, 17, 40, 86, 3, 49, 56, 73, 34, 30, 46, 28, 71, 68, 63, 63, 19, 80, 91, 12, 27, 89, 57, 81, 98, 28, 28, 66, 21, 33, 92, 82, 96, 50, 3, 80, 5, 48, 83, 52, 52, 64, 53, 27, 77, 35, 97, 73, 97, 8, 7, 64, 99, 21, 60, 68, 88, 11, 22, 93, 2, 42, 84, 67, 63, 28, 48, 69, 60, 13

In [ ]:
import csv

with open('output.csv', 'w', newline='') as csvfile:
    # Defining the CSV writer with column name for the second column only
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['', 'Category'])  # Writing the header with an unnamed first column and 'Category' as second

    # Writing the index and corresponding element from correct_list to the CSV
    for i, index in enumerate(preds):
      index = find_correct_index(incorrect_list, correct_list, index)
      csvwriter.writerow([i, correct_list[index]])

In [ ]:
!zip -r /content/file.zip /content/output.csv

  adding: content/output.csv (deflated 78%)


In [ ]:
shutil.move('/content/output.csv', '/content/drive/MyDrive')

'/content/drive/MyDrive/output.csv'